In [1]:
import pandas as pd
import numpy as np
import pandasql as psql

### Read and Filter Covid Cases

In [2]:
cases = pd.read_csv("conposcovidloc.csv")
cases = cases[(cases['Accurate_Episode_Date'] >= '2020-07-01') & (cases['Accurate_Episode_Date'] < '2020-11-01')]
cases = cases[cases['Reporting_PHU_City'].isin(['Mississauga', 'Oakville', 'Newmarket','Whitby','Toronto','Ottawa'])]
cases.reset_index(drop=True, inplace =True)

In [3]:
Patient_dimension = pd.read_csv("Patient_dimension.csv")

### Read PHU Location Dimension

In [4]:
PHU_Location_dimension = pd.read_csv("PHU_Location_dimension.csv")

In [5]:
fact = cases.merge(Patient_dimension[['Patient_ID', 'Patient_surrogate_key']], how = 'left',
                left_on = 'Row_ID', right_on = 'Patient_ID').drop('Patient_ID', axis = 1)
fact.head()

,Row_ID,Accurate_Episode_Date,Case_Reported_Date,Test_Reported_Date,Specimen_Date,Age_Group,Client_Gender,Case_AcquisitionInfo,Outcome1,Outbreak_Related,Reporting_PHU_ID,Reporting_PHU,Reporting_PHU_Address,Reporting_PHU_City,Reporting_PHU_Postal_Code,Reporting_PHU_Website,Reporting_PHU_Latitude,Reporting_PHU_Longitude,Patient_surrogate_key
0,91,2020-07-02,2020-07-03,2020-07-03,2020-07-02,30s,MALE,CC,Resolved,NaN,2253,Peel Public Health,7120 Hurontario Street,Mississauga,L5W 1N4,www.peelregion.ca/health/,43.647471,-79.708893,0
1,163,2020-07-10,2020-07-11,2020-07-11,2020-07-10,50s,MALE,CC,Resolved,Yes,2253,Peel Public Health,7120 Hurontario Street,Mississauga,L5W 1N4,www.peelregion.ca/health/,43.647471,-79.708893,1
2,203,2020-07-07,2020-07-09,2020-07-09,2020-07-07,40s,FEMALE,CC,Resolved,NaN,2253,Peel Public Health,7120 Hurontario Street,Mississauga,L5W 1N4,www.peelregion.ca/health/,43.647471,-79.708893,2
3,215,2020-07-01,2020-07-07,2020-07-07,2020-07-06,40s,MALE,OB,Resolved,NaN,2253,Peel Public Health,7120 Hurontario Street,Mississauga,L5W 1N4,www.peelregion.ca/health/,43.647471,-79.708893,3
4,217,2020-07-10,2020-07-11,2020-07-11,2020-07-10,50s,FEMALE,NO KNOWN EPI LINK,Resolved,NaN,2253,Peel Public Health,7120 Hurontario Street,Mississauga,L5W 1N4,www.peelregion.ca/health/,43.647471,-79.708893,4


### Add Surrogate Keys for PHU_Location

In [6]:
fact = fact.merge(PHU_Location_dimension[['PHU_Location_ID', 'PHU_Location_surrogate_key']], how = 'left',
                 left_on = 'Reporting_PHU_ID', right_on = 'PHU_Location_ID').drop('PHU_Location_ID', axis = 1)

In [7]:
fact.columns

Index(['Row_ID', 'Accurate_Episode_Date', 'Case_Reported_Date',
       'Test_Reported_Date', 'Specimen_Date', 'Age_Group', 'Client_Gender',
       'Case_AcquisitionInfo', 'Outcome1', 'Outbreak_Related',
       'Reporting_PHU_ID', 'Reporting_PHU', 'Reporting_PHU_Address',
       'Reporting_PHU_City', 'Reporting_PHU_Postal_Code',
       'Reporting_PHU_Website', 'Reporting_PHU_Latitude',
       'Reporting_PHU_Longitude', 'Patient_surrogate_key',
       'PHU_Location_surrogate_key'],
      dtype='object')

In [8]:
fact = fact[['Row_ID', 'Accurate_Episode_Date', 'Case_Reported_Date', 'Test_Reported_Date', 'Specimen_Date','Outcome1','Reporting_PHU','Reporting_PHU_City', 'Patient_surrogate_key','PHU_Location_surrogate_key']]

### Check unique Cities to merge with Weather Dimension

In [9]:
fact.Reporting_PHU_City.unique()

array(['Mississauga', 'Oakville', 'Newmarket', 'Whitby', 'Toronto',
       'Ottawa'], dtype=object)

### Read Weather Dimension

In [10]:
Weather_dimension_with_dates = pd.read_csv("Weather_dimension_with_dates.csv")
Weather_dimension_with_dates.head()

,Weather_surrogate_key,Station_Name,Accurate_Episode_Date,Daily_High_Temperature,Daily_Low_Temperature,Precipitation
0,0,TORONTO INTL A,2020-07-01,29.9,18.9,0.0
1,1,TORONTO INTL A,2020-07-02,35.5,21.1,0.0
2,2,TORONTO INTL A,2020-07-03,32.4,22.3,0.0
3,3,TORONTO INTL A,2020-07-04,32.0,20.2,0.0
4,4,TORONTO INTL A,2020-07-05,34.0,18.5,0.0


### Mutate Station_Name Column in fact table to facilitate merging

In [11]:
fact['Station_Name'] = np.where(fact['Reporting_PHU_City'] == 'Ottawa', 'OTTAWA CDA', 'TORONTO INTL A')

In [12]:
fact.Station_Name.unique()

array(['TORONTO INTL A', 'OTTAWA CDA'], dtype=object)

### Add Surrogate Keys for Weather

In [13]:
fact = fact.merge(Weather_dimension_with_dates[['Weather_surrogate_key', 'Station_Name', 'Accurate_Episode_Date']], how = 'left',
                 left_on = ['Station_Name', 'Accurate_Episode_Date'], right_on = ['Station_Name', 'Accurate_Episode_Date']).drop(['Station_Name'], axis = 1)

In [14]:
fact.head()

,Row_ID,Accurate_Episode_Date,Case_Reported_Date,Test_Reported_Date,Specimen_Date,Outcome1,Reporting_PHU,Reporting_PHU_City,Patient_surrogate_key,PHU_Location_surrogate_key,Weather_surrogate_key
0,91,2020-07-02,2020-07-03,2020-07-03,2020-07-02,Resolved,Peel Public Health,Mississauga,0,0,1
1,163,2020-07-10,2020-07-11,2020-07-11,2020-07-10,Resolved,Peel Public Health,Mississauga,1,0,9
2,203,2020-07-07,2020-07-09,2020-07-09,2020-07-07,Resolved,Peel Public Health,Mississauga,2,0,6
3,215,2020-07-01,2020-07-07,2020-07-07,2020-07-06,Resolved,Peel Public Health,Mississauga,3,0,0
4,217,2020-07-10,2020-07-11,2020-07-11,2020-07-10,Resolved,Peel Public Health,Mississauga,4,0,9


### Read Mobility Dimension

In [15]:
Mobility_dimension_with_dates = pd.read_csv("Mobility_dimension_with_dates.csv")

In [16]:
Mobility_dimension_with_dates.Subregion.unique()

array(['Ottawa Division', 'Regional Municipality of Durham',
       'Regional Municipality of Halton', 'Regional Municipality of Peel',
       'Regional Municipality of York', 'Toronto Division'], dtype=object)

### Mutate Subregion Column in fact table to facilitate merging with Mobility

In [17]:
fact.Reporting_PHU.unique()

array(['Peel Public Health', 'Halton Region Health Department',
       'York Region Public Health Services',
       'Durham Region Health Department', 'Toronto Public Health',
       'Ottawa Public Health'], dtype=object)

In [18]:
fact['Subregion'] = fact['Reporting_PHU']
fact['Subregion'].replace({'Peel Public Health':'Regional Municipality of Peel', 
                           'Halton Region Health Department':'Regional Municipality of Halton',                           
                           'York Region Public Health Services':'Regional Municipality of York',                          
                           'Durham Region Health Department':'Regional Municipality of Durham',                            
                           'Toronto Public Health':'Toronto Division',                           
                           'Ottawa Public Health':'Ottawa Division'
                           }, inplace = True)
fact.head()

,Row_ID,Accurate_Episode_Date,Case_Reported_Date,Test_Reported_Date,Specimen_Date,Outcome1,Reporting_PHU,Reporting_PHU_City,Patient_surrogate_key,PHU_Location_surrogate_key,Weather_surrogate_key,Subregion
0,91,2020-07-02,2020-07-03,2020-07-03,2020-07-02,Resolved,Peel Public Health,Mississauga,0,0,1,Regional Municipality of Peel
1,163,2020-07-10,2020-07-11,2020-07-11,2020-07-10,Resolved,Peel Public Health,Mississauga,1,0,9,Regional Municipality of Peel
2,203,2020-07-07,2020-07-09,2020-07-09,2020-07-07,Resolved,Peel Public Health,Mississauga,2,0,6,Regional Municipality of Peel
3,215,2020-07-01,2020-07-07,2020-07-07,2020-07-06,Resolved,Peel Public Health,Mississauga,3,0,0,Regional Municipality of Peel
4,217,2020-07-10,2020-07-11,2020-07-11,2020-07-10,Resolved,Peel Public Health,Mississauga,4,0,9,Regional Municipality of Peel


### Add Surrogate Keys for Mobility

In [19]:
fact = fact.merge(Mobility_dimension_with_dates[['Mobility_surrogate_key', 'Subregion', 'Accurate_Episode_Date']], how = 'left',
                 left_on = ['Subregion', 'Accurate_Episode_Date'], right_on = ['Subregion', 'Accurate_Episode_Date']).drop(['Subregion'], axis = 1)

In [20]:
fact.head()

,Row_ID,Accurate_Episode_Date,Case_Reported_Date,Test_Reported_Date,Specimen_Date,Outcome1,Reporting_PHU,Reporting_PHU_City,Patient_surrogate_key,PHU_Location_surrogate_key,Weather_surrogate_key,Mobility_surrogate_key
0,91,2020-07-02,2020-07-03,2020-07-03,2020-07-02,Resolved,Peel Public Health,Mississauga,0,0,1,370
1,163,2020-07-10,2020-07-11,2020-07-11,2020-07-10,Resolved,Peel Public Health,Mississauga,1,0,9,378
2,203,2020-07-07,2020-07-09,2020-07-09,2020-07-07,Resolved,Peel Public Health,Mississauga,2,0,6,375
3,215,2020-07-01,2020-07-07,2020-07-07,2020-07-06,Resolved,Peel Public Health,Mississauga,3,0,0,369
4,217,2020-07-10,2020-07-11,2020-07-11,2020-07-10,Resolved,Peel Public Health,Mississauga,4,0,9,378


### Mutate Column for Resolved, Unresolved and Fatal Measures respectively

In [21]:
fact.Outcome1.unique()

array(['Resolved', 'Fatal', 'Not Resolved'], dtype=object)

In [22]:
fact['Resolved'] = np.where(fact['Outcome1'] == 'Resolved', 1, 0)
fact['Unresolved'] = np.where(fact['Outcome1'] == 'Not Resolved', 1, 0)
fact['Fatal'] = np.where(fact['Outcome1'] == 'Fatal', 1, 0)

In [23]:
fact.head()

,Row_ID,Accurate_Episode_Date,Case_Reported_Date,Test_Reported_Date,Specimen_Date,Outcome1,Reporting_PHU,Reporting_PHU_City,Patient_surrogate_key,PHU_Location_surrogate_key,Weather_surrogate_key,Mobility_surrogate_key,Resolved,Unresolved,Fatal
0,91,2020-07-02,2020-07-03,2020-07-03,2020-07-02,Resolved,Peel Public Health,Mississauga,0,0,1,370,1,0,0
1,163,2020-07-10,2020-07-11,2020-07-11,2020-07-10,Resolved,Peel Public Health,Mississauga,1,0,9,378,1,0,0
2,203,2020-07-07,2020-07-09,2020-07-09,2020-07-07,Resolved,Peel Public Health,Mississauga,2,0,6,375,1,0,0
3,215,2020-07-01,2020-07-07,2020-07-07,2020-07-06,Resolved,Peel Public Health,Mississauga,3,0,0,369,1,0,0
4,217,2020-07-10,2020-07-11,2020-07-11,2020-07-10,Resolved,Peel Public Health,Mississauga,4,0,9,378,1,0,0


In [24]:
fact = fact.drop(['Reporting_PHU','Reporting_PHU_City', 'Outcome1'], axis = 1)
fact.head()

,Row_ID,Accurate_Episode_Date,Case_Reported_Date,Test_Reported_Date,Specimen_Date,Patient_surrogate_key,PHU_Location_surrogate_key,Weather_surrogate_key,Mobility_surrogate_key,Resolved,Unresolved,Fatal
0,91,2020-07-02,2020-07-03,2020-07-03,2020-07-02,0,0,1,370,1,0,0
1,163,2020-07-10,2020-07-11,2020-07-11,2020-07-10,1,0,9,378,1,0,0
2,203,2020-07-07,2020-07-09,2020-07-09,2020-07-07,2,0,6,375,1,0,0
3,215,2020-07-01,2020-07-07,2020-07-07,2020-07-06,3,0,0,369,1,0,0
4,217,2020-07-10,2020-07-11,2020-07-11,2020-07-10,4,0,9,378,1,0,0


### Read Date Dimension and convert Full_Date to Pandas Date Format

In [25]:
Date_dimension = pd.read_csv("Date_dimension.csv")
Date_dimension['Full_Date'] = pd.to_datetime(Date_dimension['Full_Date'])
Date_dimension.head()

,Date_surrogate_key,Full_Date,Week_Day_Name,Day,Month,Year,Day_Of_Year,Day_Of_Week,Week_In_Year,Quarter,Is_Weekend,Is_Week_Day,Is_Holiday,Holiday_Name,Season
0,100,2020-03-02,Monday,2,3,2020,62,0,10,1,0,1,0,NaN,Spring
1,101,2020-03-03,Tuesday,3,3,2020,63,1,10,1,0,1,0,NaN,Spring
2,102,2020-03-04,Wednesday,4,3,2020,64,2,10,1,0,1,0,NaN,Spring
3,103,2020-03-05,Thursday,5,3,2020,65,3,10,1,0,1,0,NaN,Spring
4,104,2020-03-06,Friday,6,3,2020,66,4,10,1,0,1,0,NaN,Spring


### Find Minimum and Maximum date in general - To generate Date Dimension (See preprocess_date.ipynb)

In [26]:
fact['Accurate_Episode_Date'] = pd.to_datetime(fact['Accurate_Episode_Date'])

print(fact['Accurate_Episode_Date'].min(),fact['Accurate_Episode_Date'].max())

2020-07-01 00:00:00 2020-10-31 00:00:00


In [27]:
fact['Test_Reported_Date'] = pd.to_datetime(fact['Test_Reported_Date'])

print(fact['Test_Reported_Date'].min(),fact['Test_Reported_Date'].max())

2020-03-31 00:00:00 2021-02-21 00:00:00


In [28]:
fact['Case_Reported_Date'] = pd.to_datetime(fact['Case_Reported_Date'])

print(fact['Case_Reported_Date'].min(), fact['Case_Reported_Date'].max())

2020-03-31 00:00:00 2021-02-17 00:00:00


In [29]:
fact['Specimen_Date'] = pd.to_datetime(fact['Specimen_Date'])

print(fact['Specimen_Date'].min(),fact['Specimen_Date'].max())

2020-03-02 00:00:00 2021-01-15 00:00:00


In [30]:
fact[fact['Test_Reported_Date'] == '2021-02-21']

,Row_ID,Accurate_Episode_Date,Case_Reported_Date,Test_Reported_Date,Specimen_Date,Patient_surrogate_key,PHU_Location_surrogate_key,Weather_surrogate_key,Mobility_surrogate_key,Resolved,Unresolved,Fatal
2309,22974,2020-09-10,2020-09-10,2021-02-21,2020-09-09,2309,0,71,440,1,0,0


### Fill NaN Dates

We assume and notice a trend in the dataset that Test Reported date in general can be taken as same date that the case was reported and specimen date can be taken as 2 days. We estimate it takes on average 2 days to get results/report based on the data.

In [31]:
fact.Test_Reported_Date.fillna(fact.Case_Reported_Date, inplace=True)

In [32]:
fact.Specimen_Date.fillna(fact.Test_Reported_Date -  pd.to_timedelta(2, unit='d'), inplace=True)
fact.head()

,Row_ID,Accurate_Episode_Date,Case_Reported_Date,Test_Reported_Date,Specimen_Date,Patient_surrogate_key,PHU_Location_surrogate_key,Weather_surrogate_key,Mobility_surrogate_key,Resolved,Unresolved,Fatal
0,91,2020-07-02,2020-07-03,2020-07-03,2020-07-02,0,0,1,370,1,0,0
1,163,2020-07-10,2020-07-11,2020-07-11,2020-07-10,1,0,9,378,1,0,0
2,203,2020-07-07,2020-07-09,2020-07-09,2020-07-07,2,0,6,375,1,0,0
3,215,2020-07-01,2020-07-07,2020-07-07,2020-07-06,3,0,0,369,1,0,0
4,217,2020-07-10,2020-07-11,2020-07-11,2020-07-10,4,0,9,378,1,0,0


### Add Surrogate Keys for dates

In [33]:
fact = fact.merge(Date_dimension[['Full_Date', 'Date_surrogate_key']], how = 'left',
                 left_on = 'Accurate_Episode_Date', right_on = 'Full_Date').drop('Full_Date', axis = 1)
fact.rename(columns={'Date_surrogate_key':'Onset_Date_surrogate_key'}, inplace=True)

In [34]:
fact = fact.merge(Date_dimension[['Full_Date', 'Date_surrogate_key']], how = 'left',
                 left_on = 'Case_Reported_Date', right_on = 'Full_Date').drop('Full_Date', axis = 1)
fact.rename(columns={'Date_surrogate_key':'Reported_Date_surrogate_key'}, inplace=True)

In [35]:
fact = fact.merge(Date_dimension[['Full_Date', 'Date_surrogate_key']], how = 'left',
                 left_on = 'Test_Reported_Date', right_on = 'Full_Date').drop('Full_Date', axis = 1)
fact.rename(columns={'Date_surrogate_key':'Test_Date_surrogate_key'}, inplace=True)

In [36]:
fact = fact.merge(Date_dimension[['Full_Date', 'Date_surrogate_key']], how = 'left',
                 left_on = 'Specimen_Date', right_on = 'Full_Date').drop('Full_Date', axis = 1)
fact.rename(columns={'Date_surrogate_key':'Specimen_Date_surrogate_key'}, inplace=True)

In [37]:
fact.head()

,Row_ID,Accurate_Episode_Date,Case_Reported_Date,Test_Reported_Date,Specimen_Date,Patient_surrogate_key,PHU_Location_surrogate_key,Weather_surrogate_key,Mobility_surrogate_key,Resolved,Unresolved,Fatal,Onset_Date_surrogate_key,Reported_Date_surrogate_key,Test_Date_surrogate_key,Specimen_Date_surrogate_key
0,91,2020-07-02,2020-07-03,2020-07-03,2020-07-02,0,0,1,370,1,0,0,222,223,223,222
1,163,2020-07-10,2020-07-11,2020-07-11,2020-07-10,1,0,9,378,1,0,0,230,231,231,230
2,203,2020-07-07,2020-07-09,2020-07-09,2020-07-07,2,0,6,375,1,0,0,227,229,229,227
3,215,2020-07-01,2020-07-07,2020-07-07,2020-07-06,3,0,0,369,1,0,0,221,227,227,226
4,217,2020-07-10,2020-07-11,2020-07-11,2020-07-10,4,0,9,378,1,0,0,230,231,231,230


### Read Special_Measures Dimension

In [38]:
Special_Measures_dimension = pd.read_csv("Special_Measures_dimension.csv")

In [39]:
Special_Measures_dimension = Special_Measures_dimension.loc[Special_Measures_dimension['Title'].isin(['Stage 3', 'Stage 2', 'Stage 3 Modified', 'Stage 2 Modified'])]

In [40]:
Special_Measures_dimension.head()

,Special_Measures_surrogate_key,Title,Description,Tag_1,Tag_2,Start_date,End_date
0,111,Stage 3,Nearly all businesses and public spaces reopen...,Businesses open,Less restrictions,7/17/2020,9/24/2020
7,118,Stage 2 Modified,Closure of indoor dining at restaurants and ba...,Non-essential business restrictions,Second wave lockdown,10/10/2020,11/16/2020
8,119,Stage 3 Modified,Non-essential businesses limited such as resta...,Non-essential business restrictions,Second wave lockdown,9/25/2020,10/9/2020
14,125,Stage 2,Some non-essential businesses reopen such as h...,Businesses open,Less restrictions,6/12/2020,7/16/2020


In [41]:
Special_Measures_dimension['Start_date'] = pd.to_datetime(Special_Measures_dimension['Start_date'])

Special_Measures_dimension['End_date'] = pd.to_datetime(Special_Measures_dimension['End_date'])


query =''' 
        select Row_ID, Special_Measures_surrogate_key 
        from Special_Measures_dimension sm join fact f
        on f.Accurate_Episode_Date between sm.Start_Date and sm.End_date 
        '''

Special_Measures_dimension = psql.sqldf(query)

Special_Measures_dimension

,Row_ID,Special_Measures_surrogate_key
0,800,111
1,2815,111
2,5661,111
3,6274,111
4,6276,111
...,...,...
36821,293701,125
36822,294185,125
36823,294852,125
36824,294993,125


In [42]:
fact = fact.merge(Special_Measures_dimension[['Row_ID','Special_Measures_surrogate_key']], how = 'left',
              left_on = 'Row_ID', right_on = 'Row_ID')

In [43]:
fact = fact.loc[:, ['Onset_Date_surrogate_key',
                    'Reported_Date_surrogate_key',
                    'Test_Date_surrogate_key', 
                    'Specimen_Date_surrogate_key',
                    'Patient_surrogate_key',
                    'PHU_Location_surrogate_key', 
                    'Mobility_surrogate_key',
                    'Special_Measures_surrogate_key',
                    'Weather_surrogate_key',
                    'Resolved', 
                    'Unresolved', 
                    'Fatal'
                    ]]

fact

,Onset_Date_surrogate_key,Reported_Date_surrogate_key,Test_Date_surrogate_key,Specimen_Date_surrogate_key,Patient_surrogate_key,PHU_Location_surrogate_key,Mobility_surrogate_key,Special_Measures_surrogate_key,Weather_surrogate_key,Resolved,Unresolved,Fatal
0,222,223,223,222,0,0,370,125,1,1,0,0
1,230,231,231,230,1,0,378,125,9,1,0,0
2,227,229,229,227,2,0,375,125,6,1,0,0
3,221,227,227,226,3,0,369,125,0,1,0,0
4,230,231,231,230,4,0,378,125,9,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
36821,307,313,313,310,36821,3,209,119,86,1,0,0
36822,310,314,314,312,36822,2,581,119,89,1,0,0
36823,341,350,350,348,36823,2,612,118,120,1,0,0
36824,333,335,335,329,36824,4,727,118,112,0,0,1


In [44]:
fact.to_csv('Fact_table.csv',index = False, header=True)